In [1]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load Q&A dataset
conn = sqlite3.connect("Auto_Reply_v3.db")
queries = [
    'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews WHERE "Developer Reply Text" IS NOT NULL',
    'SELECT "translation" AS Review, "Reply" FROM "Auto Reply Hadith" WHERE "Reply" IS NOT NULL',
    'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews WHERE "Reply" IS NOT NULL',
    'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews_Al_Quran WHERE "Reply" IS NOT NULL'
]

dfs = [pd.read_sql_query(q, conn) for q in queries]
df = pd.concat(dfs, ignore_index=True)
conn.close()

/home/tahlilmahfuz/thesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
def create_faiss_index(df):
    """Creates FAISS index for fast similarity search."""
    embeddings = model.encode(df["Review"].tolist(), convert_to_numpy=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, embeddings

# Build FAISS index
index, embeddings = create_faiss_index(df)

In [4]:
# Function to retrieve the best match
def retrieve_reply(query, index, df, top_k=1):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    return df.iloc[indices[0][0]]['Reply']

In [5]:
# Example usage
query = "Very Good App"
response = retrieve_reply(query, index, df)
print("Auto Reply:", response)

Auto Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!


In [6]:
# Example usage
query = "Bad App"
response = retrieve_reply(query, index, df)
print("Auto Reply:", response)

Auto Reply: Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran.


In [7]:
# Example usage
query = "Very Bad App"
response = retrieve_reply(query, index, df)
print("Auto Reply:", response)

Auto Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
